In [2]:
import json, glob, os
import pandas as pd

model = "log=Masked_Face_woclip+BgNoHead+shadow_cfg=Masked_Face_woclip+BgNoHead+shadow.yaml"
eval_file = glob.glob(f'/data/mint/robustness_for_evaluation/{model}/scores/*.json')
pdict = {'shape':{}, 'pose':{}, 'exp':{}, 'cam':{}}
for ef in eval_file:
    p, lvl = ef.split('/')[-1].split('128_')[-1].split('_validset')[0].split('_')
    # print(p, lvl)
    with open(ef, 'r') as f:
        score = json.load(f)
    pdict[p].update({float(lvl): score['eval_score']})

nns = {'mse':{'mean':0.007652629632502794, 'sd':0.0072924005798995495},
       'lpips':{'mean':0.07889879494905472, 'sd':0.03286045417189598},
       'dssim':{'mean':0.670, 'sd':0.0276},
       'ssim':{'mean':0, 'sd':0},
       }

score_dict = {}
for score in ['mse', 'lpips', 'dssim', 'ssim']:
    score_dict[score] = {}
    for param in pdict.keys():
        score_dict[score]['noise_lvl'] = [0]
        score_dict[score][f'{param}_mean'] = [nns[score]['mean']]
        score_dict[score][f'{param}_sd'] = [nns[score]['sd']]
        # print(f"### PARAMS : {param} ###")
        for n_lvl in sorted(pdict[param].keys()):
            mean, sd = pdict[param][n_lvl][score].split('+-')
            score_dict[score]['noise_lvl'].append(n_lvl)
            score_dict[score][f'{param}_mean'].append('{0:.7f}'.format(float(mean)))
            score_dict[score][f'{param}_sd'].append('{0:.7f}'.format(float(sd)))

    # create a excel writer object
    score_dict[score] = pd.DataFrame(score_dict[score]).T

# print(score_dict)
with pd.ExcelWriter("./probust_score.xlsx") as writer:
    for score in ['mse', 'lpips', 'dssim', 'ssim']:
        score_dict[score].to_excel(writer, sheet_name=score, header=False)
        

# Generate in Latex

In [37]:
def gen_latex(df):
    df = df.T
    print(df)
    for p in ['shape', 'pose', 'exp', 'cam']:
        txt = f"{p[0].upper()}" + p[1:]
        for row in range(len(df.index)):
            mu = df[f'{p}_mean'].iloc[row]
            sd = df[f'{p}_sd'].iloc[row]
            txt += " & ${{{0:.4f}}} \pm{{{0:.3f}}}".format(float(mu), float(sd))
        txt += " \\\\"
        print(txt)
        
    
gen_latex(score_dict['mse'])
gen_latex(score_dict['lpips'])

# "Shape & $1.02\pm{0.03}$ & $1.03\pm{0.04} $ & $1.05\pm{0.04}$ & $1.09\pm{0.38} $  & $1.15\pm{0.39}$ & $1.20\pm{0.42}$ & $0.53\pm{0.02}$ & $0.64\pm{0.02}$ \\"

  noise_lvl shape_mean   shape_sd  pose_mean    pose_sd   exp_mean     exp_sd  \
0       0.0   0.007653   0.007292   0.007653   0.007292   0.007653   0.007292   
1       0.1  0.0076608  0.0073002  0.0083348  0.0074366  0.0076572  0.0072881   
2       0.5  0.0077378  0.0073319  0.0176599  0.0161407  0.0077692  0.0073126   
3       1.0  0.0080143  0.0074582  0.0275486  0.0236689  0.0081109  0.0074605   
4       1.5  0.0085261  0.0078265  0.0335364  0.0286158  0.0086875  0.0078697   
5       2.0  0.0091948  0.0083332  0.0383574  0.0327778  0.0093576  0.0083964   
6       3.0  0.0108936  0.0096016  0.0419042  0.0343622  0.0106156  0.0092804   
7       5.0  0.0141744  0.0117883  0.0471219  0.0387570  0.0131764  0.0111162   

    cam_mean     cam_sd  
0   0.007653   0.007292  
1  0.0080560  0.0074413  
2  0.0160041  0.0147199  
3  0.0292976  0.0263397  
4  0.0383345  0.0331841  
5  0.0445977  0.0374268  
6  0.0515525  0.0419022  
7  0.0544075  0.0424934  
Shape & ${0.0077} \pm{0.008} & ${0.0

In [52]:
print(score_dict['mse'].T)

import plotly.graph_objects as go
import numpy as np
fig = go.Figure(data=go.Scatter(x=score_dict['mse']['noise_lvl'].values, y=score_dict['mse']['shape_mean'].values, mode='markers'))
fig.show()

  noise_lvl shape_mean   shape_sd  pose_mean    pose_sd   exp_mean     exp_sd  \
0       0.0   0.007653   0.007292   0.007653   0.007292   0.007653   0.007292   
1       0.1  0.0076608  0.0073002  0.0083348  0.0074366  0.0076572  0.0072881   
2       0.5  0.0077378  0.0073319  0.0176599  0.0161407  0.0077692  0.0073126   
3       1.0  0.0080143  0.0074582  0.0275486  0.0236689  0.0081109  0.0074605   
4       1.5  0.0085261  0.0078265  0.0335364  0.0286158  0.0086875  0.0078697   
5       2.0  0.0091948  0.0083332  0.0383574  0.0327778  0.0093576  0.0083964   
6       3.0  0.0108936  0.0096016  0.0419042  0.0343622  0.0106156  0.0092804   
7       5.0  0.0141744  0.0117883  0.0471219  0.0387570  0.0131764  0.0111162   

    cam_mean     cam_sd  
0   0.007653   0.007292  
1  0.0080560  0.0074413  
2  0.0160041  0.0147199  
3  0.0292976  0.0263397  
4  0.0383345  0.0331841  
5  0.0445977  0.0374268  
6  0.0515525  0.0419022  
7  0.0544075  0.0424934  


KeyError: 'noise_lvl'